## **Step 1: Setup and Install Required Libraries**



In [1]:
!pip install tensorflow tensorflow-datasets


### We’ll need TensorFlow for building the model and tensorflow_datasets to load the Tiny Shakespeare dataset.

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os

## **Step 2: Load and Explore the Dataset**

We use the Tiny Shakespeare dataset which contains lines of text from Shakespeare's plays.

In [3]:
dataset, info = tfds.load('tiny_shakespeare', with_info=True, as_supervised=False)
text = next(iter(dataset['train']))['text'].numpy().decode('utf-8')
print(text[:500])  # Preview the data

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/incomplete.CO2AUB_1.0.0/tiny_shakespeare-train.tfrecord*.…

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/incomplete.CO2AUB_1.0.0/tiny_shakespeare-validation.tfrec…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/incomplete.CO2AUB_1.0.0/tiny_shakespeare-test.tfrecord*..…

Dataset tiny_shakespeare downloaded and prepared to /root/tensorflow_datasets/tiny_shakespeare/1.0.0. Subsequent calls will reuse this data.
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


### The dataset contains lines from Shakespeare’s plays — around 100,000 characters. We’ll decode it and take a quick look at the first few lines to make sure it’s loaded correctly.

## **Step 3: Text Preprocessing**

### We need to convert characters to integers for training.

In [4]:
vocab = sorted(set(text))
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

### Before feeding the text into the model, we need to convert each character to a number. This is called tokenization. We also store the reverse mapping to convert numbers back to characters later.

## **Step 4: Create Input-Target Sequences**

In [5]:
seq_length = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

### We're training the model to predict the next character, given a sequence. So for every 100 characters, the target is the 101st one.

## **Step 5: Prepare Data for Training**

In [6]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

### To make training smoother and faster:

*   We shuffle the data.
*   Group it into batches of 64.
*   Use prefetching to load data in the background while training.



## **Step 6: Build the Model**

### Using an embedding layer, LSTM, and dense output.


In [7]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    inputs = tf.keras.Input(batch_shape=(batch_size, None), name="input_layer")
    x = tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
    x = tf.keras.layers.LSTM(
        rnn_units,
        return_sequences=True,
        stateful=True,
        recurrent_initializer='glorot_uniform'
    )(x)
    outputs = tf.keras.layers.Dense(vocab_size)(x)

    return tf.keras.Model(inputs, outputs)

In [8]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=BATCH_SIZE)

### Here’s what’s inside the model:

*   An Embedding layer to turn characters into dense vectors.
*   An LSTM layer to learn the sequence patterns.
*   A Dense output layer to predict the next character from the vocabulary.





## **Step 7: Define Loss and Compile Model**


In [9]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

### We use sparse categorical cross-entropy because we’re dealing with integer labels. Adam is used as the optimizer — it generally works well out of the box.

## **Step 8: Train the Model**

In [10]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 16s 68ms/step - loss: 2.9153
Epoch 2/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 13s 69ms/step - loss: 1.8958
Epoch 3/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 20s 70ms/step - loss: 1.6306
Epoch 4/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 13s 71ms/step - loss: 1.5004
Epoch 5/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 13s 72ms/step - loss: 1.4207
Epoch 6/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - loss: 1.3628
Epoch 7/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 22s 74ms/step - loss: 1.3205
Epoch 8/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 14s 74ms/step - loss: 1.2833
Epoch 9/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 20s 72ms/step - loss: 1.2473
Epoch 10/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 13s 72ms/step - loss: 1.2101


### We’ll train for 10 epochs and save the model weights after each one so we don’t lose progress if training gets interrupted.

## **Step 9: Load the Trained Model for Inference**

In [14]:
# Rebuild the model for inference with batch_size=1
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# ✅ Manually get the latest .weights.h5 file
import glob
latest_ckpt = sorted(glob.glob(os.path.join(checkpoint_dir, "*.weights.h5")))[-1]
print("Loading weights from:", latest_ckpt)

# ✅ Load weights directly from the .weights.h5 file
model.load_weights(latest_ckpt)

# ✅ Build the model input shape
model.build(tf.TensorShape([1, None]))


Loading weights from: ./training_checkpoints/ckpt_9.weights.h5


### Now we rebuild the model for inference with a batch size of 1 and load the latest saved weights. This setup is needed to generate text one character at a time.

## **Step 10: Generate Text**

In [17]:
def generate_text(model, start_string):
    num_generate = 1000
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    temperature = 1.0

    # ✅ Reset states of the LSTM layer only
    model.layers[2].reset_states()

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# Try generating text
print(generate_text(model, start_string="ROMEO: "))

ROMEO: COndou a bride:
Because be slale, when art thou canst not let him bade this name,
We see how the load of circumstance?
Hear not her an unsar hat force how he prose.
Is all the youngest eyes of heating fear
Before lia died: therefore, be not to bless it off,
Like enemies, because of such wate in company.

LEONTES:
Know nothing but said means earth:
Inscife from the temper. We send you,
if he delivers that knicks at that must?

JULIET:
God geld her writes.

GLOUCESTER:

QUEEN ELIZABETH:
Ay, suck from honour; but I say, yet use all ded,
Say what heart, of the sun
indeed; fast or blood upon them, and let me weep theme,
When, there attendy on 'cound carried?

All:
And, sir, sir,
He hath was an enemies; here shall art to thive,
There was both Tracious for a mirdering kingness.

DORCET:
Thou hast stunn'd himself saw to his manitage would
knd in his waboldy titly as we stood
Hath scrugg'd the duke of Rusingal England,
Not collece them between: yet his fure his langer
That thou need when

### The generate_text function takes a starting string and generates 1000 characters after that. The temperature value controls how random the output is — lower means more predictable, higher means more creative.